**water shortage likelihood prediction**

Import libraries

In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler

Load and view aquastat data set

In [165]:
df = pd.read_csv('https://mda-project-poland.s3.eu-west-3.amazonaws.com/ultimate+aquastat+(flat).csv')  # will be used for Time series analysis
df_GDP = pd.read_csv('https://mda-project-poland.s3.eu-west-3.amazonaws.com/GDP.csv')
df_2018 = df.loc[df['Year'] == 2018]  # will be used for classification

In [166]:
table_2018 = pd.pivot_table(data=df_2018,values='Value',index='Area',columns='Variable Name')

# drop undernourishment and add GDP  
table_2018 = table_2018.drop('Prevalence of undernourishment (3-year average)', axis=1)  # TODO download correct data from AQUASTAT instead of dropping here
df_GDP_2018 = df_GDP.loc[df_GDP['Year'] == 2018] 
table_GDP_2018 = pd.pivot_table(data=df_GDP_2018,values='Value',index='Area',columns='Variable Name')

table_2018 = table_2018.merge(table_GDP_2018, how='inner', on='Area')

table_2018.head() 

Variable Name,Agricultural water withdrawal as % of total renewable water resources,"Agriculture, value added (% GDP)",Average Temperature,Environmental Flow Requirements,GDP per capita,"Industry, value added to GDP",Long-term average annual precipitation in volume,MDG 7.5. Freshwater withdrawal as % of total renewable water resources,National Rainfall Index (NRI),Population density,SDG 6.4.1. Industrial Water Use Efficiency,SDG 6.4.1. Irrigated Agriculture Water Use Efficiency,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,"Services, value added to GDP",Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Gross Domestic Product (GDP)
Area,,,,,,,,,,,,,,,,,,,,
Afghanistan,30.613807,22.440899,14.55,28.29,563.983834,4.800000e+09,213.48522,31.045461,185.0,56.937048,31.220059,0.102132,57.683901,0.935917,54.757019,1.060000e+10,55.3,1757.509385,548.072294,1.798697e+10
Argentina,3.187483,6.097129,14.99,515.80,11601.890080,1.070000e+11,1643.21640,4.301333,1062.0,15.954953,32.596217,0.152694,62.519941,13.244872,10.456664,3.010000e+11,99.1,19752.418500,851.646091,5.198717e+11
Australia,2.444073,2.121597,22.58,243.30,58689.490190,3.020000e+11,4133.81148,2.354494,717.1,3.216309,91.638244,0.418891,405.626370,70.079929,4.657865,1.010000e+12,100.0,19760.502710,697.308780,1.458953e+12
Azerbaijan,32.406633,5.212381,14.16,12.03,4729.900727,2.400000e+10,38.71020,35.088681,459.7,114.890727,50.317916,0.159418,46.604281,3.870952,53.729300,1.680000e+10,87.0,3485.086793,1222.870974,4.711248e+10
Bolivia (Plurinational State of),0.334495,11.481924,20.80,396.60,3592.084429,9.720000e+09,1258.97268,0.363763,1503.0,10.334379,281.746033,0.248811,140.699591,13.711080,1.177001,2.060000e+10,90.0,50558.691150,183.913845,4.028765e+10


In [167]:
table = pd.pivot_table(data=df,values='Value',index=['Area','Year'],columns='Variable Name')



In [168]:
df=pd.DataFrame(table.to_records())

In [169]:
df

,Area,Year,Agricultural water withdrawal as % of total renewable water resources,"Agriculture, value added (% GDP)",Average Temperature,Environmental Flow Requirements,GDP per capita,"Industry, value added to GDP",Long-term average annual precipitation in volume,MDG 7.5. Freshwater withdrawal as % of total renewable water resources,...,Prevalence of undernourishment (3-year average),SDG 6.4.1. Industrial Water Use Efficiency,SDG 6.4.1. Irrigated Agriculture Water Use Efficiency,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,"Services, value added to GDP",Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita
0,Afghanistan,1992.0,35.498101,54.207106,12.61,28.290,248.518890,5.530000e+08,213.48522,35.910480,...,29.5,25.311413,0.092143,10.237171,0.240400,63.337788,9.950000e+08,21.3,4510.012947,1619.567311
1,Afghanistan,1997.0,31.427856,62.416768,12.86,28.290,164.092893,4.220000e+08,213.48522,31.852952,...,45.6,9.091408,0.085820,7.763338,0.194590,56.181246,6.530000e+08,25.0,3374.984489,1075.032191
2,Afghanistan,2002.0,30.613807,45.134344,13.91,28.290,194.958382,9.150000e+08,213.48522,31.045461,...,47.8,11.426285,0.074770,15.252615,0.285607,54.757019,1.470000e+09,33.8,2890.609479,899.030431
3,Afghanistan,2007.0,30.613807,35.265942,13.80,28.290,389.985586,2.600000e+09,213.48522,31.045461,...,33.3,20.815284,0.110353,26.876683,0.549846,54.757019,4.030000e+09,42.6,2410.653428,751.752659
4,Afghanistan,2012.0,30.613807,28.556142,13.25,28.290,694.885618,4.360000e+09,213.48522,31.045461,...,24.7,23.227115,0.103801,47.639557,0.770774,54.757019,1.070000e+10,51.6,2096.505623,653.786919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Zimbabwe,2002.0,16.590000,24.816602,21.78,9.348,657.209841,8.860000e+08,256.72932,21.025000,...,40.4,3.517161,0.061777,9.589117,1.641160,39.476155,4.770000e+09,79.4,1673.039553,351.756566
255,Zimbabwe,2007.0,14.650000,22.393849,21.95,9.348,522.006175,1.900000e+09,256.72932,17.850000,...,42.1,9.827918,0.044898,8.137430,1.597468,33.514833,3.100000e+09,78.4,1631.864171,291.287755
256,Zimbabwe,2012.0,13.878125,8.044518,21.84,9.348,1163.418688,3.840000e+09,256.72932,17.303125,...,41.3,32.078736,0.040073,19.655079,4.396704,32.488030,9.920000e+09,77.5,1524.953965,263.898049
257,Zimbabwe,2017.0,13.850000,8.340969,21.65,9.348,1333.395663,4.190000e+09,256.72932,16.695000,...,50.9,49.212382,0.045300,27.194410,5.209213,31.346226,1.390000e+10,76.9,1404.830298,234.543442


In [170]:
# drop undernourishment and add GDP  
df = df.drop('Prevalence of undernourishment (3-year average)', axis=1)  # TODO download correct data from AQUASTAT instead of dropping here
table_GDP = pd.pivot_table(data=df_GDP,values='Value',index=['Area','Year'],columns='Variable Name')
df_GDP=pd.DataFrame(table_GDP.to_records())
df_GDP

,Area,Year,Gross Domestic Product (GDP)
0,Afghanistan,1992.0,3.444711e+09
1,Afghanistan,1997.0,2.924589e+09
2,Afghanistan,2002.0,4.141524e+09
3,Afghanistan,2007.0,9.412162e+09
4,Afghanistan,2012.0,1.913650e+10
...,...,...,...
254,Zimbabwe,2002.0,8.215468e+09
255,Zimbabwe,2007.0,6.958295e+09
256,Zimbabwe,2012.0,1.711485e+10
257,Zimbabwe,2017.0,2.204090e+10


In [171]:
df = df.merge(df_GDP, how='inner', on=['Area','Year'])
#df.drop(['Gross Domestic Product (GDP)_y'], axis=1)
df


,Area,Year,Agricultural water withdrawal as % of total renewable water resources,"Agriculture, value added (% GDP)",Average Temperature,Environmental Flow Requirements,GDP per capita,"Industry, value added to GDP",Long-term average annual precipitation in volume,MDG 7.5. Freshwater withdrawal as % of total renewable water resources,...,SDG 6.4.1. Industrial Water Use Efficiency,SDG 6.4.1. Irrigated Agriculture Water Use Efficiency,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,"Services, value added to GDP",Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Gross Domestic Product (GDP)
0,Afghanistan,1992.0,35.498101,54.207106,12.61,28.290,248.518890,5.530000e+08,213.48522,35.910480,...,25.311413,0.092143,10.237171,0.240400,63.337788,9.950000e+08,21.3,4510.012947,1619.567311,3.444711e+09
1,Afghanistan,1997.0,31.427856,62.416768,12.86,28.290,164.092893,4.220000e+08,213.48522,31.852952,...,9.091408,0.085820,7.763338,0.194590,56.181246,6.530000e+08,25.0,3374.984489,1075.032191,2.924589e+09
2,Afghanistan,2002.0,30.613807,45.134344,13.91,28.290,194.958382,9.150000e+08,213.48522,31.045461,...,11.426285,0.074770,15.252615,0.285607,54.757019,1.470000e+09,33.8,2890.609479,899.030431,4.141524e+09
3,Afghanistan,2007.0,30.613807,35.265942,13.80,28.290,389.985586,2.600000e+09,213.48522,31.045461,...,20.815284,0.110353,26.876683,0.549846,54.757019,4.030000e+09,42.6,2410.653428,751.752659,9.412162e+09
4,Afghanistan,2012.0,30.613807,28.556142,13.25,28.290,694.885618,4.360000e+09,213.48522,31.045461,...,23.227115,0.103801,47.639557,0.770774,54.757019,1.070000e+10,51.6,2096.505623,653.786919,1.913650e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Zimbabwe,2002.0,16.590000,24.816602,21.78,9.348,657.209841,8.860000e+08,256.72932,21.025000,...,3.517161,0.061777,9.589117,1.641160,39.476155,4.770000e+09,79.4,1673.039553,351.756566,8.215468e+09
255,Zimbabwe,2007.0,14.650000,22.393849,21.95,9.348,522.006175,1.900000e+09,256.72932,17.850000,...,9.827918,0.044898,8.137430,1.597468,33.514833,3.100000e+09,78.4,1631.864171,291.287755,6.958295e+09
256,Zimbabwe,2012.0,13.878125,8.044518,21.84,9.348,1163.418688,3.840000e+09,256.72932,17.303125,...,32.078736,0.040073,19.655079,4.396704,32.488030,9.920000e+09,77.5,1524.953965,263.898049,1.711485e+10
257,Zimbabwe,2017.0,13.850000,8.340969,21.65,9.348,1333.395663,4.190000e+09,256.72932,16.695000,...,49.212382,0.045300,27.194410,5.209213,31.346226,1.390000e+10,76.9,1404.830298,234.543442,2.204090e+10


In [172]:
df.rename(columns = {'SDG 6.4.2. Water Stress':'WaterStress'}, inplace = True)

In [173]:

df['Industry, value added (% GDP)']=df['Industry, value added to GDP']/df['Gross Domestic Product (GDP)']*100
df['Services, value added (% GDP)']=df['Services, value added to GDP']/df['Gross Domestic Product (GDP)']*100


In [174]:
df['WaterStress']

0      63.337788
1      56.181246
2      54.757019
3      54.757019
4      54.757019
         ...    
254    39.476155
255    33.514833
256    32.488030
257    31.346226
258    35.405370
Name: WaterStress, Length: 259, dtype: float64

In [175]:
df.isnull().sum()

Area                                                                       0
Year                                                                       0
Agricultural water withdrawal as % of total renewable water resources      7
Agriculture, value added (% GDP)                                           0
Average Temperature                                                        4
Environmental Flow Requirements                                            0
GDP per capita                                                             0
Industry, value added to GDP                                               0
Long-term average annual precipitation in volume                           0
MDG 7.5. Freshwater withdrawal as % of total renewable water resources     5
National Rainfall Index (NRI)                                              7
Population density                                                         0
SDG 6.4.1. Industrial Water Use Efficiency                                 8

In [176]:
df.dropna(inplace=True)

In [177]:
df.columns = df.columns.str.replace(' ', '_')

In [178]:
df.columns = df.columns.str.replace('%', 'per')

In [188]:
df.columns = df.columns.str.replace(',', '')
df.columns = df.columns.str.replace('(', '')
df.columns = df.columns.str.replace(')', '')
df.columns = df.columns.str.replace('.', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


In [189]:
for col in df.columns:
    print(col)

Area
Year
Agricultural_water_withdrawal_as_per_of_total_renewable_water_resources
Agriculture__value_added_per_GDP
Average_Temperature
Environmental_Flow_Requirements
GDP_per_capita
Industry__value_added_to_GDP
Long-term_average_annual_precipitation_in_volume
MDG_75_Freshwater_withdrawal_as_per_of_total_renewable_water_resources
National_Rainfall_Index_NRI
Population_density
SDG_641_Industrial_Water_Use_Efficiency
SDG_641_Irrigated_Agriculture_Water_Use_Efficiency
SDG_641_Services_Water_Use_Efficiency
SDG_641_Water_Use_Efficiency
WaterStress
Services__value_added_to_GDP
Total_population_with_access_to_safe_drinking-water_JMP
Total_renewable_water_resources_per_capita
Total_water_withdrawal_per_capita
Gross_Domestic_Product_GDP
Industry__value_added_per_GDP
Services__value_added_per_GDP


In [195]:
md = smf.mixedlm("WaterStress ~ Year+Agriculture__value_added_per_GDP+GDP_per_capita+Industry__value_added_per_GDP+Services__value_added_per_GDP+Average_Temperature+Environmental_Flow_Requirements+SDG_641_Water_Use_Efficiency+National_Rainfall_Index_NRI", df, groups=df["Area"])
mdf = md.fit()
print(mdf.summary())

                      Mixed Linear Model Regression Results
Model:                    MixedLM         Dependent Variable:         WaterStress
No. Observations:         208             Method:                     REML       
No. Groups:               36              Scale:                      33.6603    
Min. group size:          3               Likelihood:                 -770.4194  
Max. group size:          7               Converged:                  Yes        
Mean group size:          5.8                                                    
---------------------------------------------------------------------------------
                                  Coef.   Std.Err.   z    P>|z|  [0.025   0.975] 
---------------------------------------------------------------------------------
Intercept                        -411.687  157.388 -2.616 0.009 -720.161 -103.213
Year                                0.251    0.078  3.206 0.001    0.098    0.405
Agriculture__value_added_per_GDP    0.